In [2]:
# Run to stitch the webscraped Weather.gov data together (collected in 2 batches per month).  
# input: ../data/weather_gov/weather_gov[yyyymmdd]-[yyyymmdd].csv
# output: ../data/weather_gov.csv

In [4]:
import pandas as pd
import os

In [6]:
# File contents:
# date,time,hr,temp,wind_direction,wind_speed,wind_gust,visibility_miles,weather,clouds,prcp_1_hr
# 2018-06-15,18:53,18,87,N,0,,10.00, ,FEW033 SCT080 SCT100,T
# 2018-06-15,17:53,17,82,NNW,9,,10.00,  Lt rain,SCT060 BKN100 ,T
# 2018-06-15,16:53,16,88,SE,6,,10.00,  Thunder,SCT065 BKN085 BKN150, 

In [8]:
folder = '../data/weather_gov/'
output = '../data/weather_gov.csv'

In [10]:
dtypes_on_load = {
    'date': 'string',       # preserve the format
    'time': 'string',
    'hr': 'string',         # preserve the format, with leading 0 if there
    'temp': 'string',            # convert to float
    'wind_direction': 'string',
    'wind_speed': 'int',
    'wind_gust': 'string',        # string, convert nulls to 0, then int
    'visibility_miles': 'string', # convert to float
    'weather': 'string',
    'clouds': 'string',
    'prcp_1_hr': 'string'    # Convert T (trace) prcp to 0.001 (because their measurements go to 0.01 at the lowest, and trace is less than that)
}

dtypes_on_save = {
    'date': 'string',       # preserve the format
    'time': 'string',
    'hr': 'string',         # preserve the format, with leading 0 if there
    'temp': 'float',
    'wind_direction': 'string',
    'wind_speed': 'int',
    'wind_gust': 'int',
    'wind_max': 'int',      # the max of wind_speed or wind_gust
    'visibility_miles': 'float',
    'weather': 'string',
    'clouds': 'string',
    'prcp_1_hr': 'float'    # Convert T (trace) prcp to 0.001 (because their measurements go to 0.01 at the lowest, and trace is less than that)
}

In [42]:
wg = pd.DataFrame(columns = ['date','time','hr','temp','wind_direction',
                             'wind_speed','wind_gust','wind_max','visibility_miles','weather','clouds','prcp_1_hr'])
wg = wg.astype(dtypes_on_save)   

In [44]:
# step 1 - merge!
# step 1a - loop through the files, merging them

In [46]:
files = [f for f in os.listdir(folder)]

print('starting...')

# processing file X of Y: filename.csv

x = 1
y = len(files)

for file in files :
    print(f"\rprocessing file {x} of {y}: {file}                  ", end="")
    next_df = pd.read_csv(folder + str(file), dtype=dtypes_on_load)

    next_df = next_df.replace('\xa0', '', regex = True)  # remove this nonbreaking space from anywhere

    # convert to float
    next_df = next_df.replace({'temp': {'': '-500'}})     # need to correct this later...
    next_df['temp'] = next_df['temp'].astype(float)

    # this can sometimes be like '< 0.25'
    next_df = next_df.replace({'visibility_miles': {'< ': '', '> ': ''}}, regex = True)
    next_df = next_df.replace({'visibility_miles': {'': '-500'}})     # need to correct this later
    next_df['visibility_miles'] = next_df['visibility_miles'].astype(float)

    # strip the leading whitespace from weather
    next_df['weather'] = next_df['weather'].str.strip()

    # convert nulls to 0
    next_df['wind_gust'] = next_df['wind_gust'].fillna('0')
    # then change the dtype of the column
    next_df['wind_gust'] = next_df['wind_gust'].astype(int)

    # calculate the max_wind
    next_df['max_wind'] = next_df[['wind_gust', 'wind_speed']].max(axis=1)
    
    # convert T to 0.001
    next_df = next_df.replace({'prcp_1_hr': {'T': '0.001', '':'0'}})
    # then change the dtype of the column
    next_df['prcp_1_hr'] = next_df['prcp_1_hr'].astype(float)
    
    #then merge it with the main df
    wg = pd.concat([wg, next_df])
    x += 1

print()
print('done!')

starting...
processing file 175 of 175: weather_gov20250401-20250411.csv                  
done!


In [47]:
# remove duplicates
wg = wg.drop_duplicates().reset_index(drop=True)

In [48]:
# When there are multiple records per hour?  Use the max values.
# group by date, hr, and take the max time, temp, wind_speed, wind_gust, visibility_miles, prcp_1_hr.
# Take the max of strings too?  wind_direction, weather, clouds

In [49]:
wg = wg.groupby(['date','hr'])[['time','temp','wind_speed','wind_gust','max_wind','visibility_miles','prcp_1_hr','wind_direction','weather','clouds']].max().reset_index()

In [50]:
# look for nulls

In [51]:
nulls_per_column = wg.isnull().sum()
print(nulls_per_column)

date                0
hr                  0
time                0
temp                0
wind_speed          0
wind_gust           0
max_wind            0
visibility_miles    0
prcp_1_hr           0
wind_direction      0
weather             0
clouds              0
dtype: int64


In [58]:
# look for -500 in temp and visibility_miles

In [60]:
wg[(wg['temp']==-500) | (wg['visibility_miles']==-500)]

,date,hr,time,temp,wind_speed,wind_gust,max_wind,visibility_miles,prcp_1_hr,wind_direction,weather,clouds
4263,2018-06-28,13,13:53,74.0,12,18,18.0,-500.0,0.06,WSW,,SCT013 BKN020 OVC029
26196,2020-12-30,23,23:53,43.0,17,0,17.0,-500.0,0.00,N,,BKN008 OVC024


In [62]:
# Manually adjust the two instances of -500 visibility to what is reasonable based on the 2 before and after each record
wg.loc[4261:4265]

,date,hr,time,temp,wind_speed,wind_gust,max_wind,visibility_miles,prcp_1_hr,wind_direction,weather,clouds
4261,2018-06-28,11,11:53,78.0,13,0,13.0,10.0,0.001,S,Thunder,SCT009 BKN021 OVC026
4262,2018-06-28,12,12:53,78.0,6,0,6.0,10.0,0.001,WSW,Thunder,OVC021
4263,2018-06-28,13,13:53,74.0,12,18,18.0,-500.0,0.060,WSW,,SCT013 BKN020 OVC029
4264,2018-06-28,14,14:53,73.0,10,0,10.0,10.0,0.140,SSW,"Lt rain, Mist",SCT040 OVC100
4265,2018-06-28,15,15:53,73.0,5,0,5.0,10.0,0.000,W,,FEW040 BKN120 BKN170


In [64]:
wg.loc[26194:26198]

,date,hr,time,temp,wind_speed,wind_gust,max_wind,visibility_miles,prcp_1_hr,wind_direction,weather,clouds
26194,2020-12-30,21,21:53,47.0,17,0,17.0,4.0,0.0,N,"Lt rain, Mist",OVC012
26195,2020-12-30,22,22:53,45.0,13,0,13.0,7.0,0.0,NE,,OVC007
26196,2020-12-30,23,23:53,43.0,17,0,17.0,-500.0,0.0,N,,BKN008 OVC024
26197,2020-12-31,01,01:53,41.0,9,0,9.0,5.0,0.0,N,"Lt rain, Mist",
26198,2020-12-31,02,02:53,39.0,6,0,6.0,5.0,0.0,N,"Lt rain, Mist",BKN006 OVC025


In [66]:
# adjust the first one to 10 and the 2nd one to 6
wg.loc[4263,'visibility_miles'] = 10.0
wg.loc[26196,'visibility_miles'] = 6.0

In [68]:
# save the result

In [70]:
wg.to_csv(output, index=False)